In [2]:

# In[1]:
# Read Text Data
from pyspark.sql import SparkSession
import pandas as pd
from pyspark import SparkContext
spark = SparkSession.builder.appName('text mining').getOrCreate()
data =pd.read_csv('farm-ads1.csv')  

#data = spark.read.csv("SMSSpamCollection.csv", inferSchema=True, sep='\t')
newdf = data[data.columns[0:2]]
newdf

,class,words
0,-1,ad-rheumatoid ad-arthritis ad-expert ad-tip ad...
1,-1,ad-siemen ad-water ad-remediation ad-water ad-...
2,-1,ad-symptom ad-muscle ad-weakness ad-genetic ad...
3,1,d-animal ad-animal ad-wild ad-sa ad-official a...
4,-1,ad-dr ad-enrico ad-fazzini ad-parkinson ad-dis...
...,...,...
4165,-1,ad-affordable ad-ivf ad-cost ad-efficient ad-i...
4166,1,d-mozypro ad-business ad-backup ad-affordable ...
4167,1,d-oster ad-line ad-clipper ad-oster ad-factory...
4168,-1,ad-synrevoice ad-schoolconnect ad-trust ad-aut...


In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [32]:
# CLEANED DATA 
newdf.to_csv(r'farm-ads2.csv', index = False)

In [33]:

import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()


In [35]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('text mining').getOrCreate()
data = spark.read.csv("farm-ads2.csv", inferSchema=True, header= True)
#data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
#data.show(truncate=False)

data[['words']].show()

+--------------------+
|               words|
+--------------------+
|ad-rheumatoid ad-...|
|ad-siemen ad-wate...|
|ad-symptom ad-mus...|
|d-animal ad-anima...|
|ad-dr ad-enrico a...|
|ad-ulcerative ad-...|
|ad-wellcentive ad...|
|d-free ad-raw ad-...|
|ad-north ad-shore...|
|d-world ad-finest...|
|d-vet ad-online a...|
|ad-gum ad-disease...|
|d-rabbit ad-guine...|
|ac cd rom mpn rrp...|
|ad-colitis ad-sym...|
|ad-disease ad-sig...|
|d-pygmy ad-goat a...|
|d-feed ad-supplem...|
|ad-www ad-musclew...|
|ad-farm ad-truck ...|
+--------------------+
only showing top 20 rows



In [36]:
# In[2]:
# Count number of Words in each Text
from pyspark.sql.functions import length
data = data.withColumn('length', length(data['words']))
data.show()

+-----+--------------------+------+
|class|               words|length|
+-----+--------------------+------+
|   -1|ad-rheumatoid ad-...|   252|
|   -1|ad-siemen ad-wate...|   252|
|   -1|ad-symptom ad-mus...|   106|
|   1 |d-animal ad-anima...|    77|
|   -1|ad-dr ad-enrico a...|   158|
|   -1|ad-ulcerative ad-...|   252|
|   -1|ad-wellcentive ad...|   252|
|   1 |d-free ad-raw ad-...|  1037|
|   -1|ad-north ad-shore...|   252|
|   1 |d-world ad-finest...|  1522|
|   1 |d-vet ad-online a...|  6839|
|   -1|ad-gum ad-disease...|   252|
|   1 |d-rabbit ad-guine...| 32756|
|   c |ac cd rom mpn rrp...| 14589|
|   -1|ad-colitis ad-sym...|   252|
|   -1|ad-disease ad-sig...|   252|
|   1 |d-pygmy ad-goat a...|  3420|
|   1 |d-feed ad-supplem...|  1557|
|   -1|ad-www ad-musclew...|   252|
|   -1|ad-farm ad-truck ...|   252|
+-----+--------------------+------+
only showing top 20 rows



In [37]:
# In[3]:
# Compare the lenght difference between ham and spam
data.groupby('class').mean().show()


+-----+------------------+
|class|       avg(length)|
+-----+------------------+
|   -1|234.36697722567288|
|    a|            9447.0|
|   vi|           10376.0|
|   cl|             424.0|
|   y |            9218.0|
|   b |           12907.0|
|   t | 7220.333333333333|
|   ic|            8579.0|
|   re|           16519.0|
|   pl|            8919.0|
|   ho|            8347.0|
|   n |            1703.0|
|   ia|           30435.0|
|   c | 9920.666666666666|
|   me|           20374.0|
|   p |            7321.0|
|   pe|           28409.0|
|   ar|            8863.0|
|    e|           16367.0|
|   fe|           15327.0|
+-----+------------------+
only showing top 20 rows



In [38]:
# In[4]:
# Treat TF-IDF features for each text
# TF: Term Frequency
# IDF: Inverse Document Frequency
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler

tokenizer = Tokenizer(inputCol="words", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')
final_feature = VectorAssembler(inputCols=['tf_idf', 'length'],outputCol='features')

from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,final_feature])
clean_data = data_prep_pipe.fit(data).transform(data)

clean_data.show()
clean_data.take(1)
clean_data.take(1)[0][-1]

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|               words|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|   -1|ad-rheumatoid ad-...|   252|  1.0|[ad-rheumatoid, a...|[ad-rheumatoid, a...|(43296,[245,594,7...|(43296,[245,594,7...|(43297,[245,594,7...|
|   -1|ad-siemen ad-wate...|   252|  1.0|[ad-siemen, ad-wa...|[ad-siemen, ad-wa...|(43296,[43,253,35...|(43296,[43,253,35...|(43297,[43,253,35...|
|   -1|ad-symptom ad-mus...|   106|  1.0|[ad-symptom, ad-m...|[ad-symptom, ad-m...|(43296,[49,137,45...|(43296,[49,137,45...|(43297,[49,137,45...|
|   1 |d-animal ad-anima...|    77|  0.0|[d-animal, ad-ani...|[d-animal, ad-ani...|(43296,[49,137,21...|(43296,[49,137

SparseVector(43297, {245: 1.9558, 594: 2.8721, 714: 2.76, 729: 2.7752, 895: 3.0178, 1135: 3.2607, 1340: 3.5737, 1488: 3.4839, 1708: 4.2088, 1919: 3.7408, 2329: 4.0052, 2367: 4.0184, 2914: 4.3656, 3033: 4.7806, 3037: 4.4858, 3701: 4.6983, 3736: 5.117, 3856: 4.6723, 6196: 5.3915, 7505: 5.6969, 43296: 252.0})

In [39]:
# In[5]: 
# ## Split data into training and test datasets
training, test = clean_data.randomSplit([0.6, 0.4], seed=12345)

In [ ]:
# In[3]:
# Compare the lenght difference between ham and spam
data.groupby('class').mean().show()


In [40]:
# Build Logistic Regression Model
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(featuresCol='features', labelCol='label')
logr_model = log_reg.fit(training)

results = logr_model.transform(test)
results.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [41]:
# In[10]:
# #### Confusion Matrix
from sklearn.metrics import confusion_matrix
y_true = results.select("label")
y_true = y_true.toPandas()

y_pred = results.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)
print(cnf_matrix)
print("Prediction Accuracy is ", (cnf_matrix[0,0]+cnf_matrix[1,1])/sum(sum(cnf_matrix)) )


[[804  63   0   0   0   0   1   0   1   0   0   0   0   0   0]
 [ 28 781   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   1   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
Prediction Accuracy is  0.9384251036116045
